# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

# Project - Airline AI Assistant

In [1]:
# Imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [2]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

OpenAI API Key exists and begins sk-proj-


In [3]:
system_message = "You are a helpful agent for an Airline called FlightAI. "
system_message += "Give short, courteous answer, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

## Tools

In [4]:
# Create the function for getting the ticket prices

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_prices called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [5]:
get_ticket_price("London")

Tool get_ticket_prices called for London


'$799'

In [6]:
# Create function for booking confirmation

def confirm_booking(destination_city):
    return "Booking confirmed to " + destination_city

In [7]:
confirm_booking('London')

'Booking confirmed to London'

In [8]:
# Describe price function

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a ticket with return journey to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [9]:
# Describe booking confirmation function
booking_function = {
    "name": "confirm_booking",
    "description": "Confirm the booking for the customer to the destination city. Call this whenever you need to confirm the booking, for example when a customer asks 'Could you confirm the ticket for the respective city?'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [10]:
# And this is included in a list of tools:

tools = [
    {"type": "function", "function": price_function},
    {"type": "function", "function": booking_function}
]

## Getting GenAI to use the tool

In [11]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    
    # Handle ticket price lookup
    if tool_call.function.name == "get_ticket_price":
        arguments = json.loads(tool_call.function.arguments)
        city = arguments.get('destination_city')
        price = get_ticket_price(city)
        response = {
            "role": "tool", 
            "content": json.dumps({"destination_city": city,"price": price}),
            "tool_call_id": tool_call.id
        }
        return response, city
        
    # Handle booking confirmation
    elif tool_call.function.name == "confirm_booking":
        arguments = json.loads(tool_call.function.arguments)
        city = arguments.get('destination_city')
        confirmation = confirm_booking(city)
        response = {
            "role": "tool",
            "content": json.dumps({"destination_city": city, "confirmation": confirmation}),
            "tool_call_id": tool_call.id
        }
        return response, city

In [12]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        #print(response)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)

    return response.choices[0].message.content


In [13]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Tool get_ticket_prices called for Berlin
